In [1]:
import tensorflow as tf
import numpy as np 
import torch
import random

In [2]:
def uniform_random(n, samples):
    indis = [0 for i in range(samples)]
    for i in range (samples):
        indis[i] = random.randint(int(i*2**(n*n)/samples), int((i+1)*2**(n*n)/samples))
    return indis


In [3]:
def indices ( num_indices , _range ):
    _list = [[]]
    return ( _index_gen_helper ( num_indices , _range , _list ))
def _index_gen_helper ( num_indices , _range , _list ):
    _ret = []

    for i in _list :
        for j in range ( _range ):
            _temp = i. copy ()
            _temp . extend ([ j ])
            _ret . append ( _temp )
    if num_indices == 1:
        return _ret
    else :
        return _index_gen_helper ( num_indices -1 , _range , _ret )

In [4]:
def instant_indices(num_independent_indices, i):
    inds=[0 for i in range (num_independent_indices)]
    for j in range (0, num_independent_indices):
        inds[j] = i%2
        i//=2
    return inds

print(instant_indices(4,10))

[0, 1, 0, 1]


In [27]:
#A and B are the spin up and spin down tensors

def net_2x2(A,B):
    AB=[A, B]
    C = [0 for i in range (2**4)]
    count=0
    for i ,j ,k ,l in indices(4,2):  
        C[count]=torch.einsum('zijab, zjicd, zklba, zlkdc->z', AB[i], AB[j], AB[k], AB[l])
        count+=1  
    ans=torch.stack(C)
    return tf.transpose(ans)

def single_net_2x2(A,B, sample):
    AB=[A, B]
    i = instant_indices(4, sample) 
    ans=torch.einsum('zijab, zjicd, zklba, zlkdc->z', AB[i[0]], AB[i[1]], AB[i[2]], AB[i[3]])
    return ans.reshape(A.shape[0],1)

A = torch.rand((2,2,2,2))
B = torch.rand((2,2,2,2))
C= torch.rand((2,2,2,2))
D= torch.rand((2,2,2,2))
E = torch.rand((2,2,2,2))
F=torch.rand((2,2,2,2))
G=torch.rand((2,2,2,2))

x=[A,C,E,A,A,A,A,A,A,A]
y=[B,F,G,B,B,B,B,B,B,B]

netA=torch.stack(x)
netB=torch.stack(y)
netA.requires_grad = True
netB.requires_grad = True

# print(net_2x2(netA,netB))
# print()
# z = single_net_2x2(netA,netB,0)
# print(z)
# z.mean().backward()
# print(netA.grad.data)
# print(netA)



tensor([[45.5936],
        [26.7975],
        [ 9.7320],
        [45.5936],
        [45.5936],
        [45.5936],
        [45.5936],
        [45.5936],
        [45.5936],
        [45.5936]], grad_fn=<ViewBackward>)
tensor([[[[[1.7642, 1.8975],
           [1.7076, 1.5610]],

          [[1.8707, 1.8080],
           [1.5924, 1.6014]]],


         [[[2.2390, 2.4123],
           [1.8772, 1.8623]],

          [[1.4813, 1.5832],
           [1.3784, 1.2801]]]],



        [[[[1.2887, 1.3158],
           [1.5785, 0.9033]],

          [[1.4866, 1.4882],
           [1.7021, 0.8449]]],


         [[[1.0500, 1.0479],
           [1.1867, 0.5692]],

          [[1.0434, 1.0506],
           [1.2394, 0.6641]]]],



        [[[[0.9980, 0.6737],
           [0.9154, 0.6682]],

          [[0.5412, 0.3775],
           [0.5236, 0.3614]]],


         [[[0.6169, 0.4433],
           [0.6137, 0.4165]],

          [[0.2934, 0.2159],
           [0.3109, 0.1926]]]],



        [[[[1.7642, 1.8975],
           [1.7076

In [ ]:
def net_3x3(A,B):
    AB=[A,B]
    C = [0 for i in range (2**9)]
    count=0
    for i ,j ,k ,l ,m ,n ,o ,p ,q in indices(9, 2):    
        C[count]=torch.einsum('zabjk, zcamn, zbcpq, zdelj, zfdom, zefrp, zghkl, zigno, zhiqr->z', AB[i], AB[j], AB[k], AB[l], AB[m], AB[n], AB[o], AB[p], AB[q])
        count+=1
    ans=torch.stack(C)
    return tf.transpose(ans)

def single_net_3x3(A,B, sample):
    AB=[A,B]
    i = instant_indices(9, sample)   
    ans=torch.einsum('zabjk, zcamn, zbcpq, zdelj, zfdom, zefrp, zghkl, zigno, zhiqr->z', AB[i[0]], AB[i[1]], AB[i[2]], AB[i[3]], AB[i[4]], AB[i[5]], AB[i[6]], AB[i[7]], AB[i[8]])
    return ans.reshape(A.shape[0],1)

# A = torch.rand((2,2,2,2))
# B = torch.rand((2,2,2,2))
# C= torch.rand((2,2,2,2))
# D= torch.rand((2,2,2,2))
# E = torch.rand((2,2,2,2))
# F=torch.rand((2,2,2,2))
# G=torch.rand((2,2,2,2))

# x=[A,C,E,A,A]
# y=[B,F,G,B,B]

# netA=torch.stack(x)
# netB=torch.stack(y)        
# print(net_3x3(netA,netB))

# print(single_net_3x3(netA,netB,2))


In [ ]:
def net_4x4(A,B, samples):
    AB=[A,B]
    C = [0 for i in range (samples)]
    count=0
    indis = uniform_random(4, samples)
    for i in (indis):
        CI = instant_indices(16,i) #current index
        X = torch.einsum('zabeh, zcail, zdcmp, zbdqt-> zehilmpqt', AB[CI[0]], AB[CI[1]], AB[CI[2]], AB[CI[3]])
        Y = torch.einsum('zabfe, zcaji, zdcnm, zbdrq-> zfejinmrq', AB[CI[4]], AB[CI[5]], AB[CI[6]], AB[CI[7]])
        Z = torch.einsum('zabgf, zcakj, zdcon, zbdsr-> zgfkjonsr', AB[CI[8]], AB[CI[9]], AB[CI[10]], AB[CI[11]])
        W = torch.einsum('zabhg, zcalk, zdcpo, zbdts-> zhglkpots', AB[CI[12]], AB[CI[13]], AB[CI[14]], AB[CI[15]])
        C[count] = torch.einsum('zehilmpqt, zfejinmrq, zgfkjonsr, zhglkpots->z', X, Y, Z, W)
        count+=1
    ans=torch.stack(C)
    return tf.transpose(ans)

def single_net_4x4(A,B, sample):
    AB=[A,B]
    CI = instant_indices(16, sample)  
    X = torch.einsum('zabeh, zcail, zdcmp, zbdqt-> zehilmpqt', AB[CI[0]], AB[CI[1]], AB[CI[2]], AB[CI[3]])
    Y = torch.einsum('zabfe, zcaji, zdcnm, zbdrq-> zfejinmrq', AB[CI[4]], AB[CI[5]], AB[CI[6]], AB[CI[7]])
    Z = torch.einsum('zabgf, zcakj, zdcon, zbdsr-> zgfkjonsr', AB[CI[8]], AB[CI[9]], AB[CI[10]], AB[CI[11]])
    W = torch.einsum('zabhg, zcalk, zdcpo, zbdts-> zhglkpots', AB[CI[12]], AB[CI[13]], AB[CI[14]], AB[CI[15]])
    ans = torch.einsum('zehilmpqt, zfejinmrq, zgfkjonsr, zhglkpots->z', X, Y, Z, W)
    return ans.reshape(A.shape[0],1)

# A = torch.rand((2,2,2,2))
# B = torch.rand((2,2,2,2))
# C= torch.rand((2,2,2,2))
# D= torch.rand((2,2,2,2))
# E = torch.rand((2,2,2,2))
# F=torch.rand((2,2,2,2))
# G=torch.rand((2,2,2,2))

# x=[A,C,E,A,A]
# y=[B,F,G,B,B]

# netA=torch.stack(x)
# netB=torch.stack(y)  
# print(net_4x4(netA,netB,100))
# print()
# print(single_net_4x4(netA,netB,100))

In [ ]:
def net_5x5(A,B, samples):
    AB=[A,B]
    C = [0 for i in range (samples)]
    count=0
    indis = uniform_random(5, samples)
    for i in (indis):
        CI = instant_indices(25,i) #current index
        X = torch.einsum('zabfg, zcahi, zdcjk, zedlm, zbeno -> zfghijklmno', AB[CI[0]], AB[CI[1]], AB[CI[2]], AB[CI[3]], AB[CI[4]]) 
        Y = torch.einsum('zabpf, zcaqh, zdcrj, zedsl, zbetn -> zpfqhrjsltn', AB[CI[5]], AB[CI[6]], AB[CI[7]], AB[CI[8]], AB[CI[9]])
        XY = torch.einsum('zfghijklmno, zpfqhrjsltn -> zgpiqkrmson', X, Y)
        Z = torch.einsum('zabfp, zcahq, zdcjr, zedls, zbent -> zfphqjrlsnt' , AB[CI[10]], AB[CI[11]], AB[CI[12]], AB[CI[13]], AB[CI[14]])
        XYZ = torch.einsum('zgpiqkrmson, zfphqjrlsnt -> zgfihkjmlon', XY, Z)
        U = torch.einsum('zabpf, zcaqh, zdcrj, zedsl, zbetn -> zpfqhrjsltn', AB[CI[15]], AB[CI[16]], AB[CI[17]], AB[CI[18]], AB[CI[19]])
        XYZU = torch.einsum('zgfihkjmlon, zpfqhrjsltn -> zgpiqkrmsot', XYZ, U)
        V = torch.einsum('zabgp, zcaiq, zdckr, zedms, zbeot -> zgpiqkrmsot', AB[CI[20]], AB[CI[21]], AB[CI[22]], AB[CI[23]], AB[CI[24]])
        C[count] = torch.einsum('zgpiqkrmsot, zgpiqkrmsot->z', XYZU, V)
        count+=1
    ans=torch.stack(C)
    return tf.transpose(ans)

def single_net_5x5(A,B, sample):
    AB=[A,B]
    CI = instant_indices(25,sample) #current index
    X = torch.einsum('zabfg, zcahi, zdcjk, zedlm, zbeno -> zfghijklmno', AB[CI[0]], AB[CI[1]], AB[CI[2]], AB[CI[3]], AB[CI[4]]) 
    Y = torch.einsum('zabpf, zcaqh, zdcrj, zedsl, zbetn -> zpfqhrjsltn', AB[CI[5]], AB[CI[6]], AB[CI[7]], AB[CI[8]], AB[CI[9]])
    XY = torch.einsum('zfghijklmno, zpfqhrjsltn -> zgpiqkrmson', X, Y)
    Z = torch.einsum('zabfp, zcahq, zdcjr, zedls, zbent -> zfphqjrlsnt' , AB[CI[10]], AB[CI[11]], AB[CI[12]], AB[CI[13]], AB[CI[14]])
    XYZ = torch.einsum('zgpiqkrmson, zfphqjrlsnt -> zgfihkjmlon', XY, Z)
    U = torch.einsum('zabpf, zcaqh, zdcrj, zedsl, zbetn -> zpfqhrjsltn', AB[CI[15]], AB[CI[16]], AB[CI[17]], AB[CI[18]], AB[CI[19]])
    XYZU = torch.einsum('zgfihkjmlon, zpfqhrjsltn -> zgpiqkrmsot', XYZ, U)
    V = torch.einsum('zabgp, zcaiq, zdckr, zedms, zbeot -> zgpiqkrmsot', AB[CI[20]], AB[CI[21]], AB[CI[22]], AB[CI[23]], AB[CI[24]])
    ans = torch.einsum('zgpiqkrmsot, zgpiqkrmsot->z', XYZU, V)
    return ans.reshape(A.shape[0],1)

# A = torch.rand((2,2,2,2))
# B = torch.rand((2,2,2,2))
# C= torch.rand((2,2,2,2))
# D= torch.rand((2,2,2,2))
# E = torch.rand((2,2,2,2))
# F=torch.rand((2,2,2,2))
# G=torch.rand((2,2,2,2))

# x=[A,C,E,A,A]
# y=[B,F,G,B,B]

# netA=torch.stack(x)
# netB=torch.stack(y)  
# print(net_5x5(netA,netB,100))
# print()
# print(single_net_5x5(netA,netB,100))

In [ ]:

def net_6x6(A, B, samples):
    AB=[A,B]
    C = [0 for i in range (samples)]
    count=0
    indis = uniform_random(6, samples)
    for i in (indis):
        CI = instant_indices(36,i) #current index
        X = torch.einsum('zabnh, zcaoi, zdcpj, zedqk, zferl, zbfsm -> znhoipjqkrlsm', AB[CI[0]], AB[CI[1]], AB[CI[2]], AB[CI[3]], AB[CI[4]], AB[CI[5]]) 
        Y = torch.einsum('zabtn, zcauo, zdcvp, zedwq, zfexr, zbfys -> ztnuovpwqxrys', AB[CI[6]], AB[CI[7]], AB[CI[8]], AB[CI[9]], AB[CI[10]], AB[CI[11]])
        XY = torch.einsum('znhoipjqkrlsm, ztnuovpwqxrys -> zhtiujvkwlxmy', X, Y)
        Z = torch.einsum('zabnt, zcaou, zdcpv, zedqw, zferx, zbfsy -> zntoupvqwrxsy', AB[CI[12]], AB[CI[13]], AB[CI[14]], AB[CI[15]], AB[CI[16]], AB[CI[17]])
        XYZ = torch.einsum('zhtiujvkwlxmy, zntoupvqwrxsy -> zhniojpkqlrms', XY, Z)
        U = torch.einsum('zabtn, zcauo, zdcvp, zedwq, zfexr, zbfys -> ztnuovpwqxrys', AB[CI[18]], AB[CI[19]], AB[CI[20]], AB[CI[21]], AB[CI[22]], AB[CI[23]])
        XYZU = torch.einsum('zhniojpkqlrms, ztnuovpwqxrys -> zhtiujvkwlxmy', XYZ, U)
        V = torch.einsum('zabnt, zcaou, zdcpv, zedqw, zferx, zbfsy -> zntoupvqwrxsy', AB[CI[24]], AB[CI[25]], AB[CI[26]], AB[CI[27]], AB[CI[28]], AB[CI[29]])
        XYZUV = torch.einsum('zhtiujvkwlxmy, zntoupvqwrxsy -> zhniojpkqlrms', XYZU, V)
        W = torch.einsum('zabhn, zcaio, zdcjp, zedkq, zfelr, zbfms -> zhniojpkqlrms', AB[CI[30]], AB[CI[31]], AB[CI[32]], AB[CI[33]], AB[CI[34]], AB[CI[35]])
        C[count] = torch.einsum('zhniojpkqlrms, zhniojpkqlrms->z', XYZUV, W)
        count+=1
    ans=torch.stack(C)
    return tf.transpose(ans)

def single_net_6x6(A, B, sample):
    AB=[A,B] 
    CI = instant_indices(36, sample) #current index
    X = torch.einsum('zabnh, zcaoi, zdcpj, zedqk, zferl, zbfsm -> znhoipjqkrlsm', AB[CI[0]], AB[CI[1]], AB[CI[2]], AB[CI[3]], AB[CI[4]], AB[CI[5]]) 
    Y = torch.einsum('zabtn, zcauo, zdcvp, zedwq, zfexr, zbfys -> ztnuovpwqxrys', AB[CI[6]], AB[CI[7]], AB[CI[8]], AB[CI[9]], AB[CI[10]], AB[CI[11]])
    XY = torch.einsum('znhoipjqkrlsm, ztnuovpwqxrys -> zhtiujvkwlxmy', X, Y)
    Z = torch.einsum('zabnt, zcaou, zdcpv, zedqw, zferx, zbfsy -> zntoupvqwrxsy', AB[CI[12]], AB[CI[13]], AB[CI[14]], AB[CI[15]], AB[CI[16]], AB[CI[17]])
    XYZ = torch.einsum('zhtiujvkwlxmy, zntoupvqwrxsy -> zhniojpkqlrms', XY, Z)
    U = torch.einsum('zabtn, zcauo, zdcvp, zedwq, zfexr, zbfys -> ztnuovpwqxrys', AB[CI[18]], AB[CI[19]], AB[CI[20]], AB[CI[21]], AB[CI[22]], AB[CI[23]])
    XYZU = torch.einsum('zhniojpkqlrms, ztnuovpwqxrys -> zhtiujvkwlxmy', XYZ, U)
    V = torch.einsum('zabnt, zcaou, zdcpv, zedqw, zferx, zbfsy -> zntoupvqwrxsy', AB[CI[24]], AB[CI[25]], AB[CI[26]], AB[CI[27]], AB[CI[28]], AB[CI[29]])
    XYZUV = torch.einsum('zhtiujvkwlxmy, zntoupvqwrxsy -> zhniojpkqlrms', XYZU, V)
    W = torch.einsum('zabhn, zcaio, zdcjp, zedkq, zfelr, zbfms -> zhniojpkqlrms', AB[CI[30]], AB[CI[31]], AB[CI[32]], AB[CI[33]], AB[CI[34]], AB[CI[35]])
    ans = torch.einsum('zhniojpkqlrms, zhniojpkqlrms->z', XYZUV, W)
    return ans.reshape(A.shape[0],1)

# A = torch.rand((2,2,2,2))
# B = torch.rand((2,2,2,2))
# C= torch.rand((2,2,2,2))
# D= torch.rand((2,2,2,2))
# E = torch.rand((2,2,2,2))
# F=torch.rand((2,2,2,2))
# G=torch.rand((2,2,2,2))

# x=[A,C,E,A,A]
# y=[B,F,G,B,B]

# netA=torch.stack(x)
# netB=torch.stack(y)  
# print(net_6x6(netA,netB,100))
# print()
# print(single_net_6x6(netA,netB,100))

In [ ]:
#A is a batch of spin up tensors
#B is a batch of spin down tensors

def wave_func (A, B, net, samples):
    if(net==2):
        return net_2x2(A, B)
    elif(net==3):
        return net_3x3(A, B)
    elif(net==4):
        return net_4x4(A, B, samples)
    elif(net==5):
        return net_5x5(A, B, samples)
    elif(net==6):
        return net_6x6(A, B, samples)
    else:
        return 0
        
def single_wave_func (A, B, net, sample):
    if(net==2):
        return single_net_2x2(A, B)
    elif(net==3):
        return single_net_3x3(A, B)
    elif(net==4):
        return single_net_4x4(A, B, sample)
    elif(net==5):
        return single_net_5x5(A, B, sample)
    elif(net==6):
        return single_net_6x6(A, B, sample)
    else:
        return 0